<a href="https://colab.research.google.com/github/vikxoxo/NLP/blob/master/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from os import path
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import random,re,os
import pandas as pd
from sklearn.model_selection import train_test_split
import torchtext
from torchtext import data
from torchtext import vocab
from torchtext.vocab import Vectors, GloVe
from torchtext.data import Iterator, BucketIterator
import torch.optim as optim
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, RegexpTokenizer
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
CUDA =  torch.cuda.is_available()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
CUDA

True

In [ ]:
device

device(type='cuda')

In [ ]:
if(path.isfile('sentiment labelled sentences.zip')==False):
  !wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip'

In [ ]:
if(path.isfile('sentimentanalysis')==False):
  !mkdir sentimentanalysis
!unzip -o 'sentiment labelled sentences.zip' -d sentimentanalysis 

mkdir: cannot create directory ‘sentimentanalysis’: File exists
Archive:  sentiment labelled sentences.zip
  inflating: sentimentanalysis/sentiment labelled sentences/.DS_Store  
  inflating: sentimentanalysis/__MACOSX/sentiment labelled sentences/._.DS_Store  
  inflating: sentimentanalysis/sentiment labelled sentences/amazon_cells_labelled.txt  
  inflating: sentimentanalysis/sentiment labelled sentences/imdb_labelled.txt  
  inflating: sentimentanalysis/__MACOSX/sentiment labelled sentences/._imdb_labelled.txt  
  inflating: sentimentanalysis/sentiment labelled sentences/readme.txt  
  inflating: sentimentanalysis/__MACOSX/sentiment labelled sentences/._readme.txt  
  inflating: sentimentanalysis/sentiment labelled sentences/yelp_labelled.txt  
  inflating: sentimentanalysis/__MACOSX/._sentiment labelled sentences  


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
amazon_df = pd.read_csv('sentimentanalysis/sentiment labelled sentences/amazon_cells_labelled.txt', names=['reviews', 'sentiment'], sep='\t')
imdb_df = pd.read_csv('sentimentanalysis/sentiment labelled sentences/imdb_labelled.txt',names=['reviews', 'sentiment'], sep='\t')
yelp_df = pd.read_csv('sentimentanalysis/sentiment labelled sentences/yelp_labelled.txt',names=['reviews', 'sentiment'], sep='\t')

In [ ]:
yelp_df.head()

,reviews,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
df = pd.concat([amazon_df,imdb_df,yelp_df],ignore_index=True)

In [ ]:
df.shape

(2748, 2)

In [ ]:
# # df1 = pd.DataFrame(columns=['reviews','sentiment'])
# #lowercase
df['reviews'] = df['reviews'].str.lower()
# #remove punctuation
df['reviews'] = df['reviews'].str.replace('\s+[a-zA-Z]\s+',' ')


In [ ]:
print(df['reviews'])

0       so there is no way for me to plug it in here i...
1                             good case, excellent value.
2                                  great for the jawbone.
3       tied to charger for conversations lasting more...
4                                       the mic is great.
                              ...                        
2743    i think food should have flavor and texture an...
2744                             appetite instantly gone.
2745     overall was not impressed and would not go back.
2746    the whole experience was underwhelming, and th...
2747    then, as if hadn't wasted enough of my life th...
Name: reviews, Length: 2748, dtype: object


In [ ]:
def split_train_test(df, test_size):
    train, test = train_test_split(df, test_size=test_size,random_state=100)
    return train.reset_index(drop=True), test.reset_index(drop=True)

In [ ]:
# # train_df, test_df = split_train_test(df, 0.1)
train_df, val_df = split_train_test(df, 0.3)

In [ ]:
# path = '/content/drive/My Drive/Colab Notebooks/sentiment labelled sentences/'

In [ ]:
train_df.to_csv('sentimentanalysis/train.csv', index=False)
val_df.to_csv('sentimentanalysis/valid.csv', index=False)
# test_df.to_csv('sentimentanalysis/test.csv', index=False)

In [ ]:
txt_field = data.Field(sequential=True,tokenize = 'spacy' ,include_lengths=True)
label_field = data.Field(sequential=False, use_vocab=True, pad_token=None, unk_token=None)

dataset_fields = [
    ('reviews', txt_field),
    ('sentiment', label_field)
]


In [ ]:
traindata, valdata = data.TabularDataset.splits(path='.',format='csv', train='sentimentanalysis/train.csv', validation='sentimentanalysis/valid.csv', fields=dataset_fields, skip_header=True)

In [ ]:
txt_field.build_vocab(traindata,valdata, max_size=100000, vectors = 'glove.6B.100d')
label_field.build_vocab(traindata)


In [ ]:
txt_field.vocab.vectors.shape

torch.Size([5289, 100])

In [ ]:
txt_field.vocab.vectors[txt_field.vocab.stoi['product']]
# txt_field.vocab.vectors[txt_field.vocab.itos[1]]


tensor([ 0.1280,  0.3413,  0.3311, -0.0267, -0.0227, -1.0228,  0.6519, -0.1420,
         0.2910,  0.5614, -0.1294, -0.7779, -0.0147, -0.0082,  0.1977,  0.4230,
         0.6420,  0.8920,  0.2820,  0.0382, -0.0661, -0.3985, -0.0251,  0.4593,
        -0.4563,  0.3667,  0.5693, -0.1560, -0.8231, -0.4675,  0.3595,  0.9756,
        -0.0480, -0.4706,  0.6593,  0.6621,  0.1840, -0.0525, -0.6372, -0.5337,
         0.5093, -0.5586,  0.0120,  0.0967,  0.0535,  0.2957, -0.1554, -0.4062,
        -0.5804, -0.9215,  0.6170, -0.0199, -0.1937,  0.7281,  0.0768, -1.6533,
        -0.6374, -0.0603,  1.9839,  0.1353,  0.4741, -0.1415, -0.3758,  0.1504,
         0.8950, -0.0732,  0.6373, -0.3346,  0.9764, -0.4185,  0.2639,  0.6476,
        -0.0575,  0.0053,  0.3126, -0.0048, -0.2146, -0.1834, -0.5071, -0.1076,
         0.1550,  0.1201, -1.0232,  0.5519, -1.4446, -0.3760,  0.1186, -0.7098,
        -0.1665, -0.4338,  0.2761,  0.5844, -0.3868, -0.4611,  0.2423, -0.2198,
        -0.2088, -0.3773,  1.2899,  0.13

In [ ]:
train_it, val_it = data.BucketIterator.splits(datasets=(traindata, valdata), 
                                            batch_sizes=(8,8), 
                                            sort_key=lambda x: len(x.sentiment), 
                                            device=device, 
                                            sort_within_batch=True, 
                                            repeat=False)

In [ ]:

# test_it = Iterator(testdata, batch_size=16, sort=False, sort_within_batch=False, repeat=False, device=device)

In [ ]:
len(train_it), len(val_it) #,len(test_it)

(241, 104)

In [ ]:
# batch = next(iter(train_it))
# type(batch)

In [ ]:
# batch.sentiment

In [ ]:
# batch.reviews

In [ ]:
class myLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, enforce_sorted = False)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [ ]:
VOCAB_SIZE = len(txt_field.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 32
OUTPUT_DIM = 1
N_LAYERS = 2
# EPOCH = 100
LEARNING_RATE=0.002
DROPOUT = 0.1
BIDIRECTIONAL = True
PAD_IDX = txt_field.vocab.stoi[txt_field.pad_token]

model = myLSTM(VOCAB_SIZE, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [ ]:
pretrained_embeddings = txt_field.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([5289, 100])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [ ]:
UNK_IDX = txt_field.vocab.stoi[txt_field.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])


In [ ]:
optimizer = optim.Adam(model.parameters(), lr= LEARNING_RATE)

In [ ]:
print(train_it.device)

cuda


In [ ]:
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.reviews        
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.sentiment.float())
        acc = binary_accuracy(predictions, batch.sentiment)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion): 
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.reviews
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions.float(), batch.sentiment.float())
            acc = binary_accuracy(predictions, batch.sentiment)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 20

best_valid_loss = float('inf')
best_training_accuracy = float('0')
best_validation_accuracy = float('0')
opt_epoch = 0

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model, train_it, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_it, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_validation_accuracy = valid_acc
        # best_training_loss = train_loss
        # best_training_accuracy = train_acc
        opt_epoch = epoch
        torch.save(model.state_dict(), 'sentimentanalysis/sentiment_model.pt')

    if((epoch+1)%5==0):
      print(f'Epoch: {epoch+1:02}')
      print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

print(f'\nOptimal Epoch: {opt_epoch+1:02}')
print(f'Best Validation Loss achieved: {best_valid_loss:.3f}')
print(f'Best Validation Accuracy achieved: {best_validation_accuracy*100:.2f}%')

Epoch: 05
	Train Loss: 0.038 | Train Acc: 98.81%
	 Val. Loss: 0.407 |  Val. Acc: 84.13%
Epoch: 10
	Train Loss: 0.006 | Train Acc: 99.84%
	 Val. Loss: 0.959 |  Val. Acc: 83.65%
Epoch: 15
	Train Loss: 0.001 | Train Acc: 100.00%
	 Val. Loss: 1.022 |  Val. Acc: 83.89%
Epoch: 20
	Train Loss: 0.000 | Train Acc: 100.00%
	 Val. Loss: 1.248 |  Val. Acc: 83.41%

Optimal Epoch: 01
Best Validation Loss achieved: 0.398
Best Validation Accuracy achieved: 83.05%


In [ ]:
model.load_state_dict(torch.load('sentimentanalysis/sentiment_model.pt'))

# test_loss, test_acc = evaluate(model, test_it, criterion)

# print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

<All keys matched successfully>

In [ ]:
df = pd.read_csv('livemint.csv',index_col=[0])
df1 = df['Complete Text']
df.head()
# newsdf

,News headline,Date of Publication,Link to Article,Complete Text,Sentence_score
Article_sentiment,,,,,
1,India’s IT spending to drop by 4.5% compared t...,30 Apr 2020,https://www.livemint.com/news/india/india-s-it...,IT spending in India is expected to drop by 4...,[' IT spending in India is expected to drop by...
0,Lifestyle International appoints Rishi Vasudev...,16 Mar 2020,https://www.livemint.com/companies/news/lifest...,NEW DELHI : Retail company Lifestyle Internat...,[' NEW DELHI : Retail company Lifestyle Intern...
1,Geneva International Motor Show is cancelled o...,28 Feb 2020,https://www.livemint.com/news/world/geneva-int...,MUMBAI : This year’s Geneva International Mot...,[' MUMBAI : This year’s Geneva International M...
1,Delhi's IGI Airport becomes first single-use p...,17 Feb 2020,https://www.livemint.com/news/india/delhi-s-ig...,New Delhi: GMR-led Delhi International Airpor...,"["" New Delhi: GMR-led Delhi International Airp..."
1,Sudan to hand over ex-President Omar al-Bashir...,11 Feb 2020,https://www.livemint.com/news/world/sudan-to-h...,Sudan has agreed to hand ousted autocrat Omar...,"["" Sudan has agreed to hand ousted autocrat Om..."


In [ ]:
def clean_lines(df):
  df1 = pd.DataFrame(columns=['reviews','sentiment'])
	# prepare a translation table to remove punctuation
  table = str.maketrans('', '', string.punctuation)
  for index,row in df.iterrows():
    cleaned = list()
    # tokenize on white space
    word_tokens = word_tokenize(row['reviews'])
    # convert to lower case
    word_tokens = [word.lower() for word in word_tokens]
    # remove punctuation from each token
    word_tokens = [w.translate(table) for w in word_tokens]
		# remove tokens with numbers in them
    word_tokens = [word for word in word_tokens if word.isalpha() and len(word)>0]
		# store as string
    cleaned.append(' '.join(word_tokens))
    df1 = df1.append({"reviews": " ".join(cleaned[0:]),"sentiment":  row['sentiment']}, ignore_index=True)
  return df1

In [ ]:
def test(model, iterator, criterion):
    model.eval()
    test_preds = []    
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.reviews
            predictions = model(text, text_lengths).squeeze(1)
             # predictions = predictions.data.numpy()
            # the actual outputs of the model are logits, so we need to pass these values to the sigmoid function
            pred =torch.sigmoid(predictions)
            # pred = F.softmax(predictions,dim=0)
            test_preds.append(pred.cpu().data.numpy())
            # test_preds = np.hstack(test_preds)
            # print(text)
            # scoredf = sentence_score(test_preds[0])
            score = sentence_score(test_preds[0])
            # article_score(test_preds[0])
            # print(len(test_preds[0]))
    return score

In [ ]:
def sentence_score(arr):
  score = []
  POS = 1
  NEG = 0
  # NTRL = 5
  pcount = 0
  ncount = 0
  POS_RANG = 0.5
  # NEG_RANG = 0.25
  label=0
  for i in arr:
    ele = i
    # print('\n')
    if(ele >= POS_RANG):
      score.append(POS)
      pcount+=1
    elif(ele < POS_RANG):
      score.append(NEG)
      ncount+=1
    # else:
    #   score.append(NTRL)
  if (pcount/(ncount+1)) > 1:
    label = 1
  # dfX = pd.DataFrame(score,columns=['sentence_score']) 
  # print('pcount')
  # print(pcount)
  # print('ncount')
  # print(ncount)
  # print('pos to neg ratio')
  # print(pcount/(ncount+1))
  # print(arr)
  # print(score)
  return score,label


In [ ]:
def sentence_label(df,score):
  for index,row in df.iterrows():
    s = row['reviews']
    val = score[index]
    s = s +'('+ str(val)+') '
    row['reviews'] = s
  return df

In [ ]:
for i in range(df.shape[0]):
  newsdf = pd.DataFrame(nltk.tokenize.sent_tokenize(df1[i]),columns=['reviews'])
  newsdf = newsdf[:-2]
  newsdf['sentiment']=''
  newsdf1 = clean_lines(newsdf)
  # newsdf1
  newsdf1.to_csv('sentimentanalysis/news.csv', index=False)
  testdata = data.TabularDataset(
           path="sentimentanalysis/news.csv", # the file path
           format='csv',
           skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
           fields=dataset_fields)
  news_it = Iterator(testdata, batch_size=64, shuffle=False, sort=False, sort_within_batch=False, repeat=False, device=device)
  score,fscore = test(model, news_it, criterion)
  # print(score)
  hdf = sentence_label(newsdf,score)
  # test(model, news_it, criterion)
  # newsdf['sentiment'] = score
  # finaldf = pd.concat(newsdf1,sdf,axis = 1)
  # newsdf.to_csv('sentimentanalysis/news.csv', index=False)
  # print(hdf['reviews'])
  rowlist = hdf['reviews'].to_list()
  # print(fscore)
  df['Sentence_score'][i] = rowlist
  df['Article_sentiment'][i] = fscore

TypeError: ignored

In [ ]:
df.head()

,News headline,Date of Publication,Link to Article,Complete Text,Sentence_score
Article_sentiment,,,,,
1,India’s IT spending to drop by 4.5% compared t...,30 Apr 2020,https://www.livemint.com/news/india/india-s-it...,IT spending in India is expected to drop by 4...,[' IT spending in India is expected to drop by...
0,Lifestyle International appoints Rishi Vasudev...,16 Mar 2020,https://www.livemint.com/companies/news/lifest...,NEW DELHI : Retail company Lifestyle Internat...,[' NEW DELHI : Retail company Lifestyle Intern...
1,Geneva International Motor Show is cancelled o...,28 Feb 2020,https://www.livemint.com/news/world/geneva-int...,MUMBAI : This year’s Geneva International Mot...,[' MUMBAI : This year’s Geneva International M...
1,Delhi's IGI Airport becomes first single-use p...,17 Feb 2020,https://www.livemint.com/news/india/delhi-s-ig...,New Delhi: GMR-led Delhi International Airpor...,"["" New Delhi: GMR-led Delhi International Airp..."
1,Sudan to hand over ex-President Omar al-Bashir...,11 Feb 2020,https://www.livemint.com/news/world/sudan-to-h...,Sudan has agreed to hand ousted autocrat Omar...,"["" Sudan has agreed to hand ousted autocrat Om..."


In [ ]:
df.to_csv('livemint_sentiment.csv', index=False)